In [ ]:
grou_names=['PD']*len(pd_jaco_x_list_mean)+['ET']*len(et_jaco_x_list_mean)+['NC']*len(nc_jaco_x_list_mean);
age=pd_age_list+et_age_list+nc_age_list;
group_ica_mean=pd_jaco_x_list_mean+et_jaco_x_list_mean+nc_jaco_x_list_mean;
import pandas as pd
df_1=pd.DataFrame({'group':grou_names, 'age':age, 'ica_mean': group_ica_mean})
# generate a boxplot to see the data distribution by genotypes and years. Using boxplot, we can easily detect the 
# differences between different groups
#sns.boxplot(x="group", y="age", hue="ica_mean", data=df_1, palette="Set3") 
#df_1.to_csv('data.csv')
print(df_1)


In [19]:
import pandas as pd
df_1=pd.read_csv('data.csv')
import statsmodels.api as sm
from statsmodels.formula.api import ols
# Ordinary Least Squares (OLS) model
# C(Genotype):C(years) represent interaction term
model= ols('ica_mean ~ (group) + (age) + (group):(age)', data=df_1).fit()
anova_table = sm.stats.anova_lm(model, typ=3)
model1= ols('ica_mean ~ (group) + (age)', data=df_1).fit()
anova_table1 = sm.stats.anova_lm(model1, typ=3)

<KeysViewHDF5 ['HDFVersion', 'ITKVersion', 'OSName', 'OSVersion', 'TransformGroup']>
<KeysViewHDF5 ['TransformType']>


In [ ]:
print(anova_table)
print(model.conf_int())
print(anova_table1)
print(model1.conf_int())

print(model.aic, model.bic)
print(model1.aic, model1.bic)
